In [ ]:
# =====================================================================
# ①  업로드 & 저장  (3개 merged_*.csv 선택 - NON-BUBBLE 데이터)
# =====================================================================
from google.colab import files
import pathlib, glob, os, warnings, re, collections
warnings.filterwarnings("ignore")

DATA_DIR = "/content/merged_csvs"
pathlib.Path(DATA_DIR).mkdir(exist_ok=True)

uploaded = files.upload()                                  # ★ 3개 파일 업로드 (NON-BUBBLE)
for n, d in uploaded.items():
    with open(f"{DATA_DIR}/{n}", "wb") as f:
        f.write(d)
print("📝 Saved:", glob.glob(f"{DATA_DIR}/*.csv"))

# =====================================================================
# ②  중복 이름 정리 + PPIACO→PPI 통일 + 필수 열 검증 (3개 파일)
# =====================================================================
import pandas as pd

raw_paths = glob.glob(os.path.join(DATA_DIR, "*.csv"))
dedup = collections.OrderedDict()
for p in sorted(raw_paths):                                # 알파벳순 → 사본(1) 뒤로
    key = re.sub(r"\s*\(\d+\)", "", os.path.basename(p))
    if key not in dedup:                                   # 원본 1개만 채택
        dedup[key] = p
paths = list(dedup.values())

# PPIACO → PPI
for p in paths:
    df_tmp = pd.read_csv(p)
    if "PPIACO" in df_tmp.columns and "PPI" not in df_tmp.columns:
        df_tmp.rename(columns={"PPIACO": "PPI"}, inplace=True)
        df_tmp.to_csv(p, index=False)

# 필수 열(GDP 제거)
need_cols  = ["Date","CPI","PPI","FEDFUNDS","DGS10","DJIA","SP500_PE"]
macro_cols = ["CPI","PPI","FEDFUNDS","DGS10"]
dow_cols   = ["DJIA","SP500_PE"]

assert len(paths) == 3, f"❗ 현재 {len(paths)}개 CSV — 정확히 3개 필요! (NON-BUBBLE 데이터)"
for p in paths:
    miss = set(need_cols) - set(pd.read_csv(p, nrows=1).columns)
    assert not miss, f"{os.path.basename(p)} 누락 열: {miss}"
print("✅ 3개 파일·필수 열 확인 완료 (NON-BUBBLE):", [os.path.basename(p) for p in paths])

# =====================================================================
# ③  라이브러리 & 데이터 전처리
# =====================================================================
!pip -q install tsaug
import numpy as np, torch, torch.nn as nn, torch.optim as optim, random
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tsaug import TimeWarp, Drift, AddNoise

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WINDOW = 24                         ## 24개월(2년) 윈도우

# CSV 합치기 (NON-BUBBLE 데이터)
df_list = []
for p in paths:
    tmp = pd.read_csv(p, parse_dates=["Date"])
    proto = os.path.basename(p).replace(".csv", "")
    tmp["Prototype"] = proto
    df_list.append(tmp)
df = (pd.concat(df_list, ignore_index=True)
        .sort_values("Date")
        .dropna(subset=need_cols)
        .reset_index(drop=True))

# 스케일링
sc_macro = StandardScaler().fit(df[macro_cols])
sc_dow   = StandardScaler().fit(df[dow_cols])
Xm_all   = sc_macro.transform(df[macro_cols]).astype("float32")
Xd_all   = sc_dow.transform(df[dow_cols]).astype("float32")
prot_codes = pd.Categorical(df["Prototype"]).codes

print(f"📊 Training prototypes (3개 NON-BUBBLE): {df['Prototype'].unique()}")
print(f"📊 Total data points: {len(df)}")

# =====================================================================
# ④  Contrastive Dataset & DataLoader
# =====================================================================
aug = TimeWarp(n_speed_change=3, max_speed_ratio=2.0) + \
      Drift(max_drift=(0,0.1)) + \
      AddNoise(scale=0.01)

class ContrastiveDataset(Dataset):
    def __init__(self, Xm, Xd, codes, win=24):  ## Default window to 24
        self.Xm, self.Xd, self.codes, self.win = Xm, Xd, codes, win
        self.starts = [i for b in np.unique(codes)
                       for i in np.where(codes==b)[0][:-win]
                       if i+win in np.where(codes==b)[0]]
    def __len__(self): return len(self.starts)
    def __getitem__(self, idx):
        s = self.starts[idx]
        anc = np.hstack([self.Xm[s:s+self.win], self.Xd[s:s+self.win]])
        alt = np.hstack([aug.augment(self.Xm[s:s+self.win]),
                         aug.augment(self.Xd[s:s+self.win])])
        return torch.tensor(anc), torch.tensor(alt)

train_ds   = ContrastiveDataset(Xm_all, Xd_all, prot_codes, WINDOW)
batch_size = min(64, len(train_ds))          # 데이터보다 크지 않게
loader     = DataLoader(train_ds, batch_size=batch_size,
                        shuffle=True, drop_last=False)
print(f"train_ds={len(train_ds)}, batch_size={batch_size}, len(loader)={len(loader)}")

# =====================================================================
# ⑤  [REVISED] Encoder + Non-Bubble Classifier Architecture
# =====================================================================
class Encoder(nn.Module):
    def __init__(self, in_dim, emb=128):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, emb, 2, bidirectional=True, batch_first=True)
        self.fc   = nn.Linear(emb*2, emb)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = torch.cat([h[-2], h[-1]], 1)
        return nn.functional.normalize(self.fc(h), dim=1)

class NonBubbleDetector(nn.Module):
    """Combined model with encoder and non-bubble probability classifier"""
    def __init__(self, in_dim, emb=128):
        super().__init__()
        self.encoder = Encoder(in_dim, emb)
        # Classification head: outputs probability of being NON-BUBBLE
        self.classifier = nn.Sequential(
            nn.Linear(emb, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()  # Ensures output is in [0, 1]
        )

    def forward(self, x):
        z = self.encoder(x)
        prob = self.classifier(z)
        return z, prob.squeeze()

    def get_non_bubble_probability(self, x):
        """Return the probability of being non-bubble"""
        with torch.no_grad():
            _, prob = self.forward(x)
        return prob

# Initialize model
model = NonBubbleDetector(in_dim=len(macro_cols)+len(dow_cols), emb=128).to(DEVICE)

# =====================================================================
# ⑥  [REVISED] Training with Contrastive + Non-Bubble Classification Loss
# =====================================================================
temperature = 0.05
def ntxent(z1, z2):
    z1 = nn.functional.normalize(z1, dim=1)
    z2 = nn.functional.normalize(z2, dim=1)
    sim = torch.mm(z1, z2.t()) / temperature
    labels = torch.arange(z1.size(0), device=z1.device)
    return nn.CrossEntropyLoss()(sim, labels)

# Binary cross-entropy for classification
bce_loss = nn.BCELoss()

opt = optim.Adam(model.parameters(), lr=3e-4)
EPOCHS = 300

print(f"\n🚀 Starting training with non-bubble detection classifier")
for ep in range(1, EPOCHS+1):
    model.train()
    tot_loss = 0

    for anc, alt in loader:
        anc, alt = anc.to(DEVICE), alt.to(DEVICE)

        # Forward pass
        z1, prob1 = model(anc)
        z2, prob2 = model(alt)

        # Contrastive loss (for representation learning)
        cont_loss = ntxent(z1, z2)

        # Classification loss
        # Since all training data are NON-BUBBLE periods, label them as 1 (non-bubble)
        non_bubble_labels = torch.ones(anc.size(0), device=DEVICE)
        class_loss = bce_loss(prob1, non_bubble_labels) + bce_loss(prob2, non_bubble_labels)

        # Combined loss
        loss = cont_loss + 0.5 * class_loss  # Weight the classification loss

        opt.zero_grad()
        loss.backward()
        opt.step()
        tot_loss += loss.item()

    if ep % 10 == 0 or ep == 1:
        print(f"Epoch {ep:03d} | loss {tot_loss/len(loader):.4f}")

# =====================================================================
# ⑦ [REVISED] Calculate SIMILARITY for New Data
# =====================================================================
def calculate_similarity(new_csv_path, window=24, aggregation='mean'):
    """
    Calculates SIMILARITY score(s) for new data based on similarity to NON-BUBBLE training data.
    A score near 0 means very dissimilar to training data;
    A score near 1 means very similar to training data.

    Args:
        new_csv_path (str): Path to the new CSV data
        window (int): Number of months for sliding window
        aggregation (str): How to aggregate multiple windows
            - 'mean': Average similarity across all windows
            - 'max': Maximum similarity across all windows
            - 'last': Similarity of the most recent window only
            - 'all': Return all window similarities

    Returns:
        float or list: Similarity score(s) in range [0, 1]
    """
    new = pd.read_csv(new_csv_path, parse_dates=["Date"]).dropna(subset=need_cols)
    assert len(new) >= window, f"❗ Input data must have at least {window} months (rows)."

    # Scale new data using the same scalers from training
    Xm = sc_macro.transform(new[macro_cols]).astype("float32")
    Xd = sc_dow.transform(new[dow_cols]).astype("float32")

    model.eval() # Set the model to evaluation mode
    similarities = []

    with torch.no_grad():
        # Generate similarity scores for all possible windows
        for t in range(len(new) - window + 1):
            seq_data = np.hstack([Xm[t:t+window], Xd[t:t+window]])
            seq = torch.tensor(seq_data).unsqueeze(0).to(DEVICE)

            # Calculate similarity directly (non-bubble probability = similarity to training data)
            similarity = model.get_non_bubble_probability(seq).cpu().item()
            similarities.append(similarity)

    # Aggregate results based on the chosen method
    if aggregation == 'mean':
        return np.mean(similarities)
    elif aggregation == 'max':
        return np.max(similarities)
    elif aggregation == 'last':
        return similarities[-1] if similarities else 0.0
    elif aggregation == 'all':
        return similarities
    else:
        raise ValueError(f"Unknown aggregation method: {aggregation}")

Saving 2001.04-2003.12.csv to 2001.04-2003.12.csv
Saving 1987.11-1995.12.csv to 1987.11-1995.12.csv
Saving 1975.01-1980.01.csv to 1975.01-1980.01.csv
📝 Saved: ['/content/merged_csvs/1987.11-1995.12.csv', '/content/merged_csvs/2001.04-2003.12.csv', '/content/merged_csvs/1975.01-1980.01.csv']
✅ 3개 파일·필수 열 확인 완료 (NON-BUBBLE): ['1975.01-1980.01.csv', '1987.11-1995.12.csv', '2001.04-2003.12.csv']
📊 Training prototypes (3개 NON-BUBBLE): ['1975.01-1980.01' '1987.11-1995.12' '2001.04-2003.12']
📊 Total data points: 174
train_ds=102, batch_size=64, len(loader)=2

🚀 Starting training with non-bubble detection classifier
Epoch 001 | loss 3.7215
Epoch 010 | loss 2.1382
Epoch 020 | loss 1.9952
Epoch 030 | loss 1.7503
Epoch 040 | loss 1.5570
Epoch 050 | loss 1.3159
Epoch 060 | loss 1.0805
Epoch 070 | loss 0.8036
Epoch 080 | loss 0.7454
Epoch 090 | loss 0.5014
Epoch 100 | loss 0.3886
Epoch 110 | loss 0.3251
Epoch 120 | loss 0.2562
Epoch 130 | loss 0.2078
Epoch 140 | loss 0.2042
Epoch 150 | loss 0.1769


In [ ]:
# =====================================================================
# ⑨  모델 패키지 저장 및 다운로드
# =====================================================================
import torch
from google.colab import files

# --- Create a dictionary to hold everything needed for inference ---
model_package = {
    'model_state_dict': model.state_dict(),
    'model_config': {
        'in_dim': len(macro_cols) + len(dow_cols),
        'emb': 128 # The embedding size used in training
    },
    'scalers': {
        'sc_macro': sc_macro,
        'sc_dow': sc_dow
    },
    'training_info': {
        'need_cols': need_cols,
        'macro_cols': macro_cols,
        'dow_cols': dow_cols,
        'window': WINDOW
    }
}

PACKAGE_SAVE_PATH = "non_bubble_model_package.pth"

# --- Save the entire package to a single file ---
torch.save(model_package, PACKAGE_SAVE_PATH)

print(f"✅ Model package saved to: {PACKAGE_SAVE_PATH}")
print("📦 Package contains: Model Weights, Config, Scalers, and Training Info.")

# --- Trigger the download in your browser ---
files.download(PACKAGE_SAVE_PATH)

✅ Model package saved to: non_bubble_model_package.pth
📦 Package contains: Model Weights, Config, Scalers, and Training Info.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>